In [1]:
# importing numpy and pandas
import numpy as np
import pandas as pd
import datetime

In [2]:
# importing employee data
emp_details = pd.read_excel("emp_details.xlsx",sheet_name="emp")
emp_details.head()

,name,biometric_id,emp_id,full_name,official_In_time,official_out_time,max late,max early
0,Abhishek,4,TIA009,Abhishek Patel,11:00:00,20:30:00,11:10:00,20:20:00
1,Aditi,20,TIA019,Aditi Chawda,10:00:00,18:30:00,10:10:00,18:20:00
2,Santosh,27,TIA028,Santosh Chouhan,09:00:00,18:00:00,09:10:00,17:50:00
3,Bantubala,12,Bantubala,Bantubala,11:30:00,15:00:00,11:40:00,14:50:00
4,Purnima,24,TIA024,Purnima Khandelwal,10:00:00,18:30:00,10:10:00,18:20:00


In [3]:
# data fiel read 
# name_of_dataFile = input("Please enter the name of attendance data file:")
df = pd.read_csv("attendance.dat",header=None)

# Data cleaning because the data is in one column spliting it and keeping only the req. two columns so make these function that extract the data in first two columns

def get_id(text):
    return text.split("\t")[0]
def get_time(text):
    return text.split("\t")[1]

# creating two columns ID and punch_Time that hold the data when the employee punch in and out

df["id"]= df[0].map(get_id)
df["punch_datetime"] = df[0].map(get_time)
df.drop(columns=[0],inplace=True)   # deleting the raw data column and keeping only two column
df['id']= df["id"].astype(int)      # conerting id column to int

df.head()


,id,punch_datetime
0,23,2025-04-01 09:17:01
1,25,2025-04-01 09:25:18
2,28,2025-04-01 09:57:22
3,24,2025-04-01 09:59:49
4,1,2025-04-01 10:50:40


In [4]:
df['punch_datetime'] = pd.to_datetime(df['punch_datetime'])
df['punch_date'] = df['punch_datetime'].dt.date
df['punch_date'] = pd.to_datetime(df['punch_date'],format="%y-%m-%d")
df['punch_time'] = df['punch_datetime'].dt.time


In [5]:
df.shape

(519, 4)

In [6]:
df = df.groupby(['punch_date','id'])['punch_time'].agg(['min',"max"]).reset_index()

In [7]:
df.rename(columns={'id':"biometric_id","min":'actual_in_time','max':'actual_out_time'},inplace=True)

In [8]:
df = pd.merge(df,emp_details,on='biometric_id')

In [9]:
df

,punch_date,biometric_id,actual_in_time,actual_out_time,name,emp_id,full_name,official_In_time,official_out_time,max late,max early
0,2025-04-01,1,10:50:40,19:02:35,Saheba,TIA004,Saheba Khan,11:00:00,18:30:00,11:10:00,18:20:00
1,2025-04-01,5,10:58:09,18:53:30,Kuldeep,TIA013,Kuldeep Chouhan,11:00:00,18:00:00,11:10:00,17:50:00
2,2025-04-01,12,11:01:31,15:06:11,Bantubala,Bantubala,Bantubala,11:30:00,15:00:00,11:40:00,14:50:00
3,2025-04-01,23,09:17:01,18:30:02,Devesh,TIA023,Devesh Patwa,09:30:00,18:00:00,09:40:00,17:50:00
4,2025-04-01,24,09:59:49,18:53:43,Purnima,TIA024,Purnima Khandelwal,10:00:00,18:30:00,10:10:00,18:20:00
...,...,...,...,...,...,...,...,...,...,...,...
263,2025-04-30,20,09:26:35,18:39:42,Aditi,TIA019,Aditi Chawda,10:00:00,18:30:00,10:10:00,18:20:00
264,2025-04-30,23,09:24:03,18:27:49,Devesh,TIA023,Devesh Patwa,09:30:00,18:00:00,09:40:00,17:50:00
265,2025-04-30,24,13:54:31,18:27:14,Purnima,TIA024,Purnima Khandelwal,10:00:00,18:30:00,10:10:00,18:20:00
266,2025-04-30,25,09:17:29,18:27:47,Arvind,TIA026,Arvind Vishwakarma,09:30:00,18:00:00,09:40:00,17:50:00


In [10]:
# importing leave data
leave_data = pd.read_excel("leave_data.xlsx",sheet_name='leave')

# selecting only necessory column
leave_data =  leave_data[['Name','Leave Start Date','Leave End Date','Half Day/ Full Day?','Number of Days',"Type of leave"]]

# creating first name column
leave_data['first name'] = leave_data.apply(lambda x: x["Name"].split(' ')[0].strip(),axis=1)

leave_data['Leave End Date'] = pd.to_datetime(leave_data['Leave End Date'], errors='coerce')
leave_data['Leave Start Date'] = pd.to_datetime(leave_data['Leave Start Date'], errors='coerce')

leave_data = leave_data[leave_data['Leave Start Date'].notna()]
leave_data.head()

,Name,Leave Start Date,Leave End Date,Half Day/ Full Day?,Number of Days,Type of leave,first name
0,Anjali Bohra,2022-03-21,2022-03-21,Full day,1.0,Casual Leave,Anjali
1,Devendra Patidar,2022-03-24,2022-03-26,Full day,3.0,Sick leave (Illness or Injury),Devendra
2,Saheba Khan,2022-03-26,2022-03-26,Full day,1.0,Exam,Saheba
3,Saheba Khan,2022-03-29,2022-03-29,Full day,1.0,Exam,Saheba
4,Saheba Khan,2022-03-31,2022-03-31,Full day,1.0,Exam,Saheba


In [11]:
# fixing the structure of leave data table to long format
expanded_data = []

for _, row in leave_data.iterrows():
    start_date = row['Leave Start Date']
    end_date = row['Leave End Date']
    
    # Generate the date range
    dates = pd.date_range(start=start_date, end=end_date)
    
    for date in dates:
        expanded_data.append({
            'Name': row['Name'],
            "first name" : row["first name"],
            'Date': date,
            'Half Day/ Full Day?': row['Half Day/ Full Day?'],
            "Type of leave" : row['Type of leave']
        })

# Create the expanded DataFrame
leave_data = pd.DataFrame(expanded_data)
leave_data.head()

,Name,first name,Date,Half Day/ Full Day?,Type of leave
0,Anjali Bohra,Anjali,2022-03-21,Full day,Casual Leave
1,Devendra Patidar,Devendra,2022-03-24,Full day,Sick leave (Illness or Injury)
2,Devendra Patidar,Devendra,2022-03-25,Full day,Sick leave (Illness or Injury)
3,Devendra Patidar,Devendra,2022-03-26,Full day,Sick leave (Illness or Injury)
4,Saheba Khan,Saheba,2022-03-26,Full day,Exam
